In [1]:
import pandas as pd
import numpy as np

## <span style='color:midnightblue'> Table 1 - table with column values colour coded (categories) </span>

<div class="alert alert-block alert-success">
The table below shows climate change results with some columns colour coded based on categorical variables in the dataframe. Note that the example below uses output from a ODH program (GetStats). This functionality will need to be included in bulum.
</div>

In [2]:
result_path='.\Plot3'

results_files=['BR_RCP85_2050_00a_TexasTWS.in_stats.csv',
               'BR_RCP85_2050_00a_YelarbonTWS.in_stats.csv',
               'BR_RCP85_2050_00a_GoondiwindiTWS.in_stats.csv',
               'BR_RCP85_2050_00a_InglewoodTWS.in_stats.csv',
               'GB_RCP85_2050_00a_StanthorpeTWS.in_stats.csv',
               'GB_RCP85_2050_00a_UnallocHS.in_stats.csv',
               'UB_RCP85_2050_00a_YarramanTWS.in_stats.csv',
               'UB_RCP85_2050_00a_ToowoombaTWS.in_stats.csv',
               'MC_RCP85_2050_00a_ChinchillaTWS.in_stats.csv']

users=['Texas TWS',  'Yelarbon TWS', 'Goondiwindi TWS', 'Inglewood TWS', 'Stanthorpe TWS', 'Unallocated HP', 'Yarraman TWS', 'Toowoomba TWS', 'Chinchilla TWS']

catchment=['Border Rivers', 'Border Rivers', 'Border Rivers', 'Border Rivers', 'Granite Belt', 'Granite Belt', 'Upper Brisbane', 'Upper Brisbane', 'Middle Condamine']

demand=[270, 106, 1800, 488, 1150, 1500, 200, 24000, 1165]

results=[]

for file in results_files:
    results.append(pd.read_csv(result_path + '\\' + file))

def rating_hist(x):
    if x >= 1:
        return 'Good'
    elif x > 0.9:
        return 'OK'
    elif x > 0.8:
        return 'Poor'
    elif x > 0.5:
        return 'Very Poor'
    else:
        return 'Extremely Poor'

def cc_trend(x):
    if x >= 10:
        return 'Strongly positive'
    elif x >= 7:
        return 'Positive'
    elif x >= 5:
        return 'Neutral'
    elif x >= 3:
        return 'Negative'
    else:
        return 'Strongly negative'

def cc_sensi(x):
    if x >= 0.5:
        return 'Extreme'
    elif x >= 0.25:
        return 'Very high'
    elif x >= 0.15:
        return 'High'
    elif x >= 0.05:
        return 'Moderate'
    else:
        return 'Low'

hist_results=[]
hist_rating=[]
gcm_results=[]

for result in results:
    hist_results.append(result.loc[0,'Annual reliability'])
    hist_rating.append(rating_hist(result.loc[0,'Annual reliability']))
    gcm_results.append(result.loc[1:11,'Annual reliability'])

gcm_maint=[]
gcm_reduc=[]
rat_trend=[]
gcm_10th=[]
gcm_90th=[]
rat_sensi=[]
i=0

for result in gcm_results:
    gcm_maint.append(np.count_nonzero(result >= hist_results[i]))
    gcm_reduc.append(np.count_nonzero(result < hist_results[i]))
    rat_trend.append(cc_trend(gcm_maint[i]))
    gcm_10th.append(np.percentile(result,10))
    gcm_90th.append(np.percentile(result,90))
    rat_sensi.append(cc_sensi(hist_results[i] - gcm_10th[i]))
    i+=1

resultsTable=pd.DataFrame({
    'Catchment':catchment,
    'Users':users,
    'Simulated_Demand':demand,
    'Historical_Reliability':hist_results,
    'Hist_Perf_Rating':hist_rating,
    'GCMs_Maintained':gcm_maint,
    'GCMs_Reduced':gcm_reduc,
    'Rating_CC_Trend':rat_trend,
    'Second_Best_GCM_Perf':gcm_90th,
    'Second_Worst_GCM_Perf':gcm_10th,
    'Rating_CC_Sensitivity':rat_sensi
})

def style_rating(v):
    if v == 'Extremely Poor' or v == 'Strongly negative' or v == 'Extreme':
        return 'color:firebrick;'
    elif v== 'Very Poor' or v == 'Negative' or v == 'Very high':
        return 'color:goldenrod;'
    elif v== 'Poor' or v == 'Neutral' or v == 'High':
        return 'color:gold;'
    elif v== 'OK' or v == 'Positive' or v == 'Moderate':
        return 'color:seagreen;'
    elif v== 'Good' or v == 'Strongly positive' or v == 'Low':
        return 'color:royalblue;'

resultsTable.style.format({
    'Historical_Reliability': '{:,.1%}'.format,
    'Simulated_Demand': '{:,}'.format,
    'Second_Best_GCM_Perf': '{:,.1%}'.format,
    'Second_Worst_GCM_Perf': '{:,.1%}'.format,
}).applymap(style_rating).hide_index()

Catchment,Users,Simulated_Demand,Historical_Reliability,Hist_Perf_Rating,GCMs_Maintained,GCMs_Reduced,Rating_CC_Trend,Second_Best_GCM_Perf,Second_Worst_GCM_Perf,Rating_CC_Sensitivity
Border Rivers,Texas TWS,270,53.8%,Very Poor,4,7,Negative,75.8%,15.2%,Very high
Border Rivers,Yelarbon TWS,106,71.2%,Very Poor,2,9,Strongly negative,78.8%,21.2%,Extreme
Border Rivers,Goondiwindi TWS,"1,800",93.9%,OK,5,6,Neutral,98.5%,81.1%,Moderate
Border Rivers,Inglewood TWS,488,81.8%,Poor,4,7,Negative,95.5%,39.4%,Very high
Granite Belt,Stanthorpe TWS,"1,150",93.1%,OK,4,7,Negative,99.2%,47.7%,Very high
Granite Belt,Unallocated HP,"1,500",98.5%,OK,5,6,Neutral,100.0%,67.7%,Very high
Upper Brisbane,Yarraman TWS,200,100.0%,Good,11,0,Strongly positive,100.0%,100.0%,Low
Upper Brisbane,Toowoomba TWS,"24,000",88.3%,Poor,5,6,Neutral,93.7%,64.9%,High
Middle Condamine,Chinchilla TWS,"1,165",100.0%,Good,7,4,Positive,100.0%,99.2%,Low


## <span style='color:midnightblue'> Table 2 - table with column values colour coded (values) </span>

<div class="alert alert-block alert-success">
The table below WASO results and highlights the number red if the objective is failed and blue if the objective is passed. Note that the example below uses output from a Queensland Hydrology program (MC5). We can start to include this functionality in bulum, but will need to check the outcomes are consistent with program provided by the Department.
</div>

In [3]:
scenarios=['03c', '03d', '03e', '03a', '03b']
result_path=".\Table2"
hp2_file='901a-hp2_'

waso_results_hp2=[]

for scenario in scenarios:
    waso_results_hp2.append(pd.read_csv(result_path + '\\' + hp2_file + scenario + '.csv'))

hp2_ann=[]
hp2_mon=[]

for result in waso_results_hp2:
    hp2_ann.append(float(result.loc[1,' Annual Reliability (%)'])/100)    
    hp2_mon.append(float(result.loc[1,'Monthly Reliability (%)'])/100)

wasoResult=pd.DataFrame({
    'Scenario':scenarios,
    'Annual supplemented WSI':hp2_ann,
    'Monthly supplemented WSI':hp2_mon
})

suppObjec=[0.95, 0.98, 0.82, 0.82]

def obj_waso_1a(val):
    if val < suppObjec[0]:
        return 'color:firebrick;'
    else:
        return 'color:royalblue;'

def obj_waso_1b(val):
    if val < suppObjec[1]:
        return 'color:firebrick;'
    else:
        return 'color:royalblue;'
    
wasoResult.style.format({
    'Annual supplemented WSI': '{:.1%}'.format, 
    'Monthly supplemented WSI': '{:,.1%}'.format
}).applymap(obj_waso_1a,subset=['Annual supplemented WSI']
           ).applymap(obj_waso_1b,subset=['Monthly supplemented WSI']).set_table_styles([dict(selector="th",props=[('max-width', '100px')])]).hide_index()

Scenario,Annual supplemented WSI,Monthly supplemented WSI
03c,55.1%,82.7%
03d,88.8%,95.6%
03e,91.6%,97.1%
03a,92.5%,98.1%
03b,95.3%,98.9%
